In [0]:
from google.colab import drive 
drive.mount('/content/gdrive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

In [0]:
window_size = 1
batch_size = 5
shuffle_buffer = 1000
start = 300000
end = 310000
num_samples = 10000

In [0]:
data_path = '/content/gdrive/My Drive/'
train_signal_file = data_path + 'train_signal.npy'
train_channels_file = data_path + 'train_channels.npy'
test_signal_file = data_path + 'test_signal.npy'

train_signal = np.load(train_signal_file)
train_channels = np.load(train_channels_file)
test_signal = np.load(test_signal_file)

valid_signal = train_signal[:,400000:]
valid_channels = train_channels[:,400000:]
train_signal = train_signal[:,:400000]
train_channels = train_channels[:,:400000]


In [0]:
x = train_signal[:,start:end].flatten()
y = train_channels[:,start:end].flatten()
valid_x = valid_signal.flatten()
valid_y = valid_channels.flatten()

model = tf.keras.models.Sequential([
  tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                      input_shape=[None]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
  tf.keras.layers.Dense(11,activation = 'softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
history = model.fit(x = x,y=y, batch_size = batch_size, shuffle = True, epochs = 5)



Epoch 1/5
20000/20000 [==============================] - 155s 8ms/step - loss: 0.4238 - acc: 0.8510
Epoch 2/5
20000/20000 [==============================] - 163s 8ms/step - loss: 0.3656 - acc: 0.8633
Epoch 3/5
20000/20000 [==============================] - 181s 9ms/step - loss: 0.3634 - acc: 0.8644
Epoch 4/5
20000/20000 [==============================] - 180s 9ms/step - loss: 0.3620 - acc: 0.8643
Epoch 5/5
20000/20000 [==============================] - 171s 9ms/step - loss: 0.3619 - acc: 0.8652


In [0]:
y_pred = model.predict(valid_x)
y_true = np.array(valid_y).astype(int)
print(y_pred[:10])
print(y_true[:10])
y_pred_1 = np.zeros(y_true.shape)
for j in range(y_pred.shape[0]):
    maxval = 0
    index = 0
    for i in range(11):
        if(y_pred[j][i] > maxval):
            maxval = y_pred[j][i]
            index = i
    y_pred_1[j] = index

#print(y_true.shape)
y_pred_1 = np.array(y_pred_1).astype(int)
y_true = np.array(y_true).astype(int)
print(f1_score(y_true, y_pred_1, average=None))
print(f1_score(y_true, y_pred_1, average='macro'))
    

[[9.9800712e-01 4.4106884e-04 6.1362999e-04 7.5754942e-04 1.7632148e-04
  8.4876507e-07 2.8061854e-06 7.2874599e-07 1.4443453e-09 3.4915395e-09
  6.4906303e-11]
 [9.9729037e-01 5.5985287e-04 7.9916563e-04 1.1128113e-03 2.3195057e-04
  1.2204741e-06 4.0004184e-06 7.0793698e-07 6.8133238e-10 3.7736871e-09
  1.3044493e-10]
 [9.9826318e-01 4.4448476e-04 5.4391072e-04 6.0925080e-04 1.3565597e-04
  7.4597853e-07 1.9982449e-06 8.6941014e-07 4.5973088e-09 4.9591899e-09
  3.6468113e-11]
 [9.9824834e-01 4.2963095e-04 5.4976717e-04 6.2515563e-04 1.4343087e-04
  7.4663149e-07 2.1501828e-06 8.2838454e-07 3.4082823e-09 4.3591433e-09
  4.0837451e-11]
 [9.9813336e-01 4.2681448e-04 5.8105052e-04 6.9223711e-04 1.6255531e-04
  7.9116143e-07 2.5265886e-06 7.5841689e-07 1.9480932e-09 3.6568433e-09
  5.3557977e-11]
 [1.4274445e-01 8.2306439e-01 4.6908688e-03 2.8205080e-02 1.2587378e-03
  3.6369627e-05 3.1173876e-08 9.1769675e-10 4.0972989e-08 8.9332204e-09
  3.4219914e-12]
 [9.9787998e-01 7.0585345e-04 6.07

In [0]:
def windowed_dataset(all_series, all_channels, window_size, batch_size, shuffle_buffer):
    
    all_dataset = np.zeros((end - start)*10*(window_size + 1))
    cnt = 0
    s = 0
    for series in all_series:
        for i in range(start,end):
            all_dataset[s:s + window_size] = series[i-window_size+1:i+1]
            s += window_size
            all_dataset[s] = all_channels[cnt][i]
            s+=1
        cnt+=1
    #print(all_dataset)
    series = np.array(all_dataset)
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=window_size+1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer).map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

dataset = windowed_dataset(train_signal, train_channels, window_size, batch_size, shuffle_buffer)

In [0]:
models = []

for k in range (10):
  x = train_signal[k].flatten()
  y = train_channels[k].flatten()
  valid_x = valid_signal[k].flatten()
  valid_y = valid_channels[k].flatten()

  model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
                        input_shape=[None]),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
    tf.keras.layers.Dense(11,activation = 'softmax')
  ])

  model.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['acc'])
  
  if(k==9 or k==4):
    history = model.fit(x = x,y=y, batch_size = batch_size, shuffle = True, epochs = 2)
  else:
    history = model.fit(x = x,y=y, batch_size = batch_size, shuffle = True, epochs = 30, steps_per_epoch=500)

  models.append(model)
  y_pred = model.predict(valid_x)
  y_true = np.array(valid_y).astype(int)
  print(y_pred[:10])
  print(y_true[:10])
  y_pred_1 = np.zeros(y_true.shape)
  for j in range(y_pred.shape[0]):
      maxval = 0
      index = 0
      for i in range(11):
          if(y_pred[j][i] > maxval):
              maxval = y_pred[j][i]
              index = i
      y_pred_1[j] = index

  #print(y_true.shape)
  y_pred_1 = np.array(y_pred_1).astype(int)
  y_true = np.array(y_true).astype(int)
  print(f1_score(y_true, y_pred_1, average=None))
  print(f1_score(y_true, y_pred_1, average='macro'))
  print(k,"   done")



Epoch 1/30
500/500 [==============================] - 4s 7ms/step - loss: 0.3021 - acc: 0.9628
Epoch 2/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0840 - acc: 0.9648
Epoch 3/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0455 - acc: 0.9804
Epoch 4/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0136 - acc: 0.9984
Epoch 5/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0114 - acc: 0.9972
Epoch 6/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0141 - acc: 0.9960
Epoch 7/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0079 - acc: 0.9976
Epoch 8/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0077 - acc: 0.9976
Epoch 9/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0103 - acc: 0.9972
Epoch 10/30
500/500 [==============================] - 4s 7ms/step - loss: 0.0073 - acc: 0.9980
Epoch 11/30
500/500 [============================

In [0]:

model_ref = [1,3,5,1,2,9,5,9,1,3,1,1,1,1,1,1,1,1,1,1]
y_pred_all = np.zeros((2000000))
for pec in range(20):
  x_test = test_signal.flatten()[pec*100000:(pec+1)*100000]
  test_pred = models[model_ref[pec]].predict(x_test)
  y_pred_1 = np.zeros(x_test.shape[0])
  for j in range(test_pred.shape[0]):
      maxval = 0
      index = 0
      for i in range(11):
          if(test_pred[j][i] > maxval):
              maxval = test_pred[j][i]
              index = i
      y_pred_1[j] = index
  y_pred_all[pec*100000:(pec+1)*100000] = y_pred_1

y_pred_all = np.array(y_pred_all).astype(int)

# from numpy import asarray
# from numpy import savetxt
# savetxt(data_path + 'my_sub.csv', y_pred_1, fmt='%i', delimiter=',')

In [0]:
# model_spec = tf.keras.models.Sequential([
#     tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                         input_shape=[None]),
#       tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
#     tf.keras.layers.Dense(11,activation = 'softmax')
#   ])

# model_spec.compile(loss='sparse_categorical_crossentropy',
#               optimizer='adam',
#               metrics=['acc'])

ranges = [4,9]
for k in ranges:

  # x_spec = train_signal[0].flatten()[:10000]
  # y_spec = train_channels[0].flatten()[:10000]

  # model_spec.fit(x = x_spec,y=y_spec, batch_size = batch_size, shuffle = True, epochs = 5)
  # x = train_signal[k].flatten()
  # y = train_channels[k].flatten()
  valid_x = valid_signal[k].flatten()
  valid_y = valid_channels[k].flatten()

  
  # model_spec.fit(x = x,y=y, batch_size = batch_size, shuffle = True, epochs = 1)
  
  y_pred = models[4].predict(valid_x)
  y_true = np.array(valid_y).astype(int)
  print(y_pred[:10])
  print(y_true[:10])
  y_pred_1 = np.zeros(y_true.shape)
  y_pred_3 = models[2].predict(valid_x)
  y_pred_1_3 = np.zeros(y_true.shape)

  for j in range(y_pred_3.shape[0]):
      
      maxval = 0
      index = 0
      for i in range(11):
          if(y_pred_3[j][i] > maxval):
              maxval = y_pred_3[j][i]
              index = i
      y_pred_1_3[j] = index

  y_pred_1_3 = np.array(y_pred_1_3).astype(int)



  for j in range(y_pred.shape[0]):
      if(valid_x[j]<-2):
        y_pred_1[j] = y_pred_1_3[j]
        continue
      maxval = 0
      index = 0
      for i in range(11):
          if(y_pred[j][i] > maxval):
              maxval = y_pred[j][i]
              index = i
      y_pred_1[j] = index

  #print(y_true.shape)
  y_pred_1 = np.array(y_pred_1).astype(int)
  y_true = np.array(y_true).astype(int)
  print(f1_score(y_true, y_pred_1, average=None))
  print(f1_score(y_true, y_pred_1, average='macro'))
  print(k,"   done")

[[3.3863498e-10 5.4909872e-08 4.0876455e-11 2.1733882e-08 3.1773880e-08
  1.6233528e-06 1.6195967e-06 4.6280331e-07 9.2853579e-06 1.1044670e-02
  9.8894227e-01]
 [5.2342808e-10 5.2049078e-08 9.8607504e-11 4.5929642e-08 7.8410956e-08
  3.8350481e-06 3.8928533e-06 1.2796088e-06 3.1577812e-05 3.2036494e-02
  9.6792275e-01]
 [3.1260416e-10 5.7655342e-08 3.2693202e-11 1.7868425e-08 2.5483491e-08
  1.3150294e-06 1.2962306e-06 3.5745677e-07 6.8242193e-06 8.4170643e-03
  9.9157310e-01]
 [4.1100864e-10 5.2466088e-08 6.2577776e-11 3.1312027e-08 4.8947634e-08
  2.4511414e-06 2.4769263e-06 7.5708220e-07 1.6756119e-05 1.8506140e-02
  9.8147124e-01]
 [1.7236941e-09 4.4582791e-08 1.3508311e-09 3.1070613e-07 8.8552468e-07
  3.4116460e-05 3.4316490e-05 1.9169438e-05 1.0072633e-03 5.3515869e-01
  4.6374518e-01]
 [1.7265590e-09 4.4145207e-08 1.3834817e-09 3.1440325e-07 8.9881752e-07
  3.4471319e-05 3.4678327e-05 1.9518615e-05 1.0354926e-03 5.4514229e-01
  4.5373228e-01]
 [1.2903619e-09 2.0517698e-08 7.49

In [0]:
batches = [5,7]

for pec in batches:
  x_test = test_signal.flatten()[pec*100000:(pec+1)*100000]
  test_pred = models[model_ref[3]].predict(x_test)
  y_pred_1 = np.zeros(x_test.shape[0])
  for j in range(test_pred.shape[0]):
      if(x_test[j]>0):
        y_pred_1[j] = y_pred_all[pec*100000 + j]
        continue

      maxval = 0
      index = 0
      for i in range(11):
          if(test_pred[j][i] > maxval):
              maxval = test_pred[j][i]
              index = i
      y_pred_1[j] = index
  y_pred_all[pec*100000:(pec+1)*100000] = y_pred_1

y_pred_all = np.array(y_pred_all).astype(int)




In [0]:
import pandas as pd
sub = pd.read_csv(data_path + 'sample_submission_ion.csv')

sub.iloc[:,1] = y_pred_all
sub.to_csv(data_path + 'my_submission_diffmo3.csv',index=False,float_format='%.4f')
print("saved the file 3")

saved the file 3


In [1]:
x_last = train_signal[9].flatten()
y_last = train_channels[9].flatten()

NameError: ignored